In [1]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import itertools

from kernels_class import *
from rbf_kernel import *
from generation import *

# Basic settings

- p = 5
- $n_{+} = 250$, $n_{-}=250$, $n' = 500$
- $\mu_{+} = (0,0,\ldots,0)$,  $\Sigma_{+} = I_p$
- $\beta = 1$
- $\mu_{-} = \beta\cdot(1,1,\ldots,1)$,  $\Sigma_{-} = I_p$



In [2]:
params = {'beta': [0.5, 1],
          'n_plus': [500],
          'n_minus': [500], 
          'n_target': [500], 
          'pi_target': [0.5],
          'p': [int(5)],
          'seed': [123]}

params.keys()

dict_keys(['beta', 'n_plus', 'n_minus', 'n_target', 'pi_target', 'p', 'seed'])

In [3]:
df_params = pd.DataFrame(itertools.product(*params.values()),columns=params.keys())

In [4]:
N = df_params.shape[0]
df_results = pd.DataFrame(np.zeros((N,3)), columns=['pi_nmr', 'pi_irp', 'var'])
df = pd.concat((df_params, df_results), axis=1)


    

In [5]:
df['p']

0    5
1    5
Name: p, dtype: int64

In [6]:
dict(df.loc[0, params.keys()])

{'beta': np.float64(0.5),
 'n_plus': np.float64(500.0),
 'n_minus': np.float64(500.0),
 'n_target': np.float64(500.0),
 'pi_target': np.float64(0.5),
 'p': np.float64(5.0),
 'seed': np.float64(123.0)}

In [14]:
df_params

,beta,n_plus,n_minus,n_target,pi_target,p,seed
0,0.5,500,500,500,0.5,5,123
1,1.0,500,500,500,0.5,5,123


In [15]:
simulation(df_params, generate_sample_Nstd_Nstd_CC, params.keys(), ['pi_nmr_u', 'pi_ipr_u', 
                                       'pi_nmr_v', 'pi_ipr_v', 
                                       'var_plug-in', 'var_explicit'])

(   beta  n_plus  n_minus  n_target  pi_target  p  seed  pi_nmr_u  pi_ipr_u  \
 0   0.5     500      500       500        0.5  5   123  0.512454  0.518780   
 1   1.0     500      500       500        0.5  5   123  0.501785  0.503776   
 
    pi_nmr_v  pi_ipr_v  var_plug-in  var_explicit  var_plug-in_target  \
 0  0.523816  0.518630     0.428251      0.412417            0.001911   
 1  0.505517  0.503797     0.170898      0.169058            0.010958   
 
    var_plug-in_plus  var_plug-in_minus  var_explicit_target  \
 0          0.000357           0.000325             0.001878   
 1          0.001042           0.001083             0.010903   
 
    var_explicit_plus  var_explicit_minus  
 0           0.000326            0.000293  
 1           0.000999            0.001040  ,
 np.float64(0.17591386302413517))

In [10]:
generate_sample_Nstd_Nstd_CC(**df.loc[0, params.keys()].to_dict())

(array([[-0.57860025,  1.65143654, -2.42667924, -0.42891263,  1.26593626],
        [-0.8667404 , -0.67888615, -0.09470897,  1.49138963, -0.638902  ],
        [-0.44398196, -0.43435128,  2.20593008,  2.18678609,  1.0040539 ],
        ...,
        [ 2.79197117, -0.50939499,  0.02085102, -1.15200246, -1.8863966 ],
        [ 0.89835491,  0.90545284, -1.37544054, -1.68979089, -0.63588161],
        [ 1.66702579, -1.48814135,  0.68179686,  0.13142663, -0.30181745]],
       shape=(500, 5)),
 array([[-0.1046016 ,  0.74498205,  0.18920493,  0.1006796 ,  0.29246025],
        [ 1.61162249,  0.71284144,  0.56895192,  1.88257879,  0.23185155],
        [ 1.93283757, -0.1758123 ,  1.03114891,  0.17689671,  0.43150372],
        ...,
        [ 1.16228775, -0.54065638,  1.45384854, -1.41018362,  1.27844784],
        [-0.5579392 , -0.62227046, -0.91863786, -1.35971515,  1.41382219],
        [-0.85383977,  0.8187635 ,  2.01110387, -0.63662678,  1.14245991]],
       shape=(500, 5)),
 array([[-1.12788175, -0

In [11]:
def simulation(c1, c2, c3, N):

    n_plus = 250
    n_minus = 250
    n_target = 500
    pi_target = 0.5

    p = 5
    beta = 1
    
    for i in tqdm(range(N)):

        p_plus, p_minus, p_target = generate_sample_Nstd_Nstd_CC(p, beta,
                                                                 n_plus, n_minus, n_target, pi_target)
        
        est_tmp = estimator_RHKS(p_target, p_plus, p_minus, kernel_params={'gamma': 1/p})
        est_tmp.estimate_pi_nrm()
        est_tmp.estimate_pi_ipr()

        res_nmr[i,j] = est_tmp.pi_nrm
        res_ipr[i,j] = est_tmp.pi_ipr

        est_tmp.compute_K2()
        est_tmp.compute_tau_plug_in()
        est_tmp.compute_tau_explicit()

        est_tmp.estimate_variance_explicit()
        est_tmp.estimate_variance_plug_in()

        res_var_plug_in[i,j] = est_tmp.var_plug_in
        res_var_explicit[i,j] = est_tmp.var_explicit

        


In [12]:
res_nmr = np.zeros((N, n2))
res_ipr = np.zeros((N, n2))
res_var_plug_in = np.zeros((N, n2))
res_var_explicit = np.zeros((N, n2))

res_estim_nmr = np.zeros((N, n2))
res_estim_ipr = np.zeros((N, n2))

delta = np.linspace(0.8, 4, n2)

for i in tqdm(range(N)):

    p_source_plus, p_source_minus, p_target = generate_sample_fig3(1)

    for j in range(n2):
        
        est_tmp = estimator_RHKS(p_target, p_source_plus, p_source_minus, kernel_params={'gamma': 1/(2*(delta[j]**2))})
        est_tmp.estimate_pi_nrm()
        est_tmp.estimate_pi_ipr()

        res_nmr[i,j] = est_tmp.pi_nrm
        res_ipr[i,j] = est_tmp.pi_ipr

        est_tmp.compute_K2()
        est_tmp.compute_tau_plug_in()
        est_tmp.compute_tau_explicit()

        est_tmp.estimate_variance_explicit()
        est_tmp.estimate_variance_plug_in()

        res_var_plug_in[i,j] = est_tmp.var_plug_in

        res_var_explicit[i,j] = est_tmp.var_explicit

        var_est, tar_est, pos_est, neg_est = variance_for_rbf(np.mean(p_source_plus, 0), np.cov(p_source_plus.T), 
                                                              np.mean(p_source_minus, 0), np.cov(p_source_plus.T), est_tmp.pi_nrm, 
                 1/(r*n_target),1/(r*n_source_plus),1/(r*n_source_minus), gamma=1/(2*(delta[j])**2))

        res_estim_nmr[i,j] = var_est

        var_est, tar_est, pos_est, neg_est = variance_for_rbf(np.mean(p_source_plus, 0), np.cov(p_source_plus.T), 
                                                              np.mean(p_source_minus, 0), np.cov(p_source_plus.T), est_tmp.pi_ipr, 
                 1/(r*n_target),1/(r*n_source_plus),1/(r*n_source_minus), gamma=1/(2*(delta[j])**2))
        res_estim_ipr[i,j] = var_est


NameError: name 'n2' is not defined